In [3]:
!pip install helper_functions

  Created wheel for helper-functions: filename=helper_functions-2.0.11-py3-none-any.whl size=7527 sha256=155d8190284f8f6b60f56220426f302f5b19086a24764472bd448342d8099cce
  Stored in directory: /Users/ashokpandey/Library/Caches/pip/wheels/33/c4/10/4f022732779bd35274968b8302709922143303ac5968466881
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25858 sha256=00a25d26edb071ddef262db334fe9a2754ad9e9ef1c5f2675a02b1a91a989538
  Stored in directory: /Users/ashokpandey/Library/Caches/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=7d8b15eefc137538031d5cda098b3f1c65887a47b63e34f911c32a832743a3a7
  Stored in directory: /Users/ashokpandey/Library/Caches/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built helper-functions imutils sklearn


In [7]:
!pip install io_helper

     |████████████████████████████████| 380 kB 7.4 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Users/ashokpandey/opt/anaconda3/envs/dev/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/wg/dbmj0v8x4k91_3hgslxt96p40000gn/T/pip-install-wvkbf2q5/psycopg2_8e063aa46f014b0db23931dcad210203/setup.py'"'"'; __file__='"'"'/private/var/folders/wg/dbmj0v8x4k91_3hgslxt96p40000gn/T/pip-install-wvkbf2q5/psycopg2_8e063aa46f014b0db23931dcad210203/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/wg/dbmj0v8x4k91_3hgslxt96p40000gn/T/pip-pip-egg-info-do72oaz_
         cwd: /private/var/folders/wg/dbmj0v8x4k91_3hgslxt96p40000gn/T/pip-install-wvkbf2q5/psycopg2_8e0

In [6]:
import sys
import requests
import datetime
import time
import zlib
import msgpack

# import files from src
sys.path.insert(0, "src")
#import helper_functions as hf
import io_helper as ioh

# define the url of the endpoint to get event data
endpoint_url = "http://api-dev.augmento.ai/v0.1/events/aggregated"

# define where we're going to save the data
path_save_data = "data/example_data"
filename_save_data = "{:s}/augmento_data.msgpack.zlib".format(path_save_data)

# define the start and end times
datetime_start = datetime.datetime(2017, 1, 1)
datetime_end = datetime.datetime(2019, 9, 25)

# initialise a store for the data we're downloading
sentiment_data = []

# define a start pointer to track multiple requests
start_ptr = 0
count_ptr = 1000

# get the data
while start_ptr >= 0:
	
	# define the parameters of the request
	params = {
		"source" : "twitter",
		"coin" : "bitcoin",
		"bin_size" : "1H",
		"count_ptr" : count_ptr,
		"start_ptr" : start_ptr,
		"start_datetime" : datetime_start.strftime("%Y-%m-%dT%H:%M:%SZ"),
		"end_datetime" : datetime_end.strftime("%Y-%m-%dT%H:%M:%SZ"),
	}
	
	# make the request
	r = requests.request("GET", endpoint_url, params=params, timeout=10)
	
	# if the request was ok, add the data and increment the start_ptr
	# else return an error
	if r.status_code == 200:
		temp_data = r.json()
		start_ptr += count_ptr
	else:
		raise Exception("api call failed with status_code {:d}".format(r.status_code))
	
	# if we didn't get any data, assume we've got all the data
	if len(temp_data) == 0:
		start_ptr = -1
	
	# extend the data store
	sentiment_data.extend(temp_data)
	
	# print the progress
	str_print = "got data from {:s} to {:s}".format(*(sentiment_data[0]["datetime"],
	                                                sentiment_data[-1]["datetime"],))
	print(str_print)
	
	# sleep
	time.sleep(2.0)

# check if the data path exists
ioh.check_path(path_save_data, create_if_not_exist=True)

# save the data
print("saving data to {:s}".format(filename_save_data))
with open(filename_save_data, "wb") as f:
	f.write(zlib.compress(msgpack.packb(sentiment_data)))


print("done!")

ModuleNotFoundError: No module named 'io_helper'